In [1]:
import numpy as np

from ipywidgets import interact
from matplotlib import pyplot as plt

%matplotlib inline

# Q1

In [2]:
def multivariate_gaussian_pdf(x, mu, det_Sigma, Sigma_inv, d=2):
    constant = 1 / ((2 * np.pi)**(d / 2) * det_Sigma**0.5)
    return constant * np.exp(-0.5 * (x - mu).T @ Sigma_inv @ (x - mu))

In [3]:
# generated synthetic data from three distinct Gaussian distributions
# true (unknown) means
mu_1_true = np.array([-4, 2])
mu_2_true = np.array([-1, 1])
mu_3_true = np.array([-4, 0])

# true (unknown) covariances
Sigma_1_true = 0.1 * np.eye(2)
Sigma_2_true = 0.2 * np.eye(2)
s = 0.8
Sigma_3_true = 0.1 * np.array([[1, s], [s, 1]])

# varying number of data points per class
n1 = 500
n2 = 1000
n3 = 250

# generate data
x_1 = np.random.multivariate_normal(mu_1_true, Sigma_1_true, size=n1)
x_2 = np.random.multivariate_normal(mu_2_true, Sigma_2_true, size=n2)
x_3 = np.random.multivariate_normal(mu_3_true, Sigma_3_true, size=n3)

# fit class proportions 
n = n1 + n2 + n3
pi_1 = n1 / n
pi_2 = n2 / n
pi_3 = n3 / n

# fit means and variances by maximum likelihood
# i.e. calculate the empirical mean and covariance for each class
mu_1 = np.mean(x_1, axis=0)
mu_2 = np.mean(x_2, axis=0)
mu_3 = np.mean(x_3, axis=0)

Sigma_1 = np.cov(x_1, rowvar=False)
Sigma_2 = np.cov(x_2, rowvar=False)
Sigma_3 = np.cov(x_3, rowvar=False)

# need the inverse and determinant of the covariance before we can use the classifier
Sigma_1_inv = np.linalg.inv(Sigma_1)
Sigma_2_inv = np.linalg.inv(Sigma_2)
Sigma_3_inv = np.linalg.inv(Sigma_3)

det_Sigma_1 = np.linalg.det(Sigma_1)
det_Sigma_2 = np.linalg.det(Sigma_2)
det_Sigma_3 = np.linalg.det(Sigma_3)

def plot_classification_behaviour(test_x_1=-2.5, test_x_2=1):
    # choose a test point to classify
    test_point = np.array([test_x_1, test_x_2])
    
    # calculate scores for each class
    # these are the Gaussian densities for each class multiplied by the class proportions
    score_class_1 = multivariate_gaussian_pdf(test_point, mu_1, det_Sigma_1, Sigma_1_inv) * pi_1
    score_class_2 = multivariate_gaussian_pdf(test_point, mu_2, det_Sigma_2, Sigma_1_inv) * pi_2
    score_class_3 = multivariate_gaussian_pdf(test_point, mu_3, det_Sigma_3, Sigma_1_inv) * pi_3
    
    # the normalizer for these scores is just their sum 
    normalizer = score_class_1 + score_class_2 + score_class_3
    
    # probabilities are given by normalizing the scores
    prob_class_1 = score_class_1 / normalizer
    prob_class_2 = score_class_2 / normalizer
    prob_class_3 = score_class_3 / normalizer
    
    # find prediction for test point
    probabilities = [prob_class_1, prob_class_2, prob_class_3]
    class_prediction = np.argmax(probabilities)
    prediction_probability = probabilities[class_prediction] 
    
    # do some plotting 
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))
    
    # plot data and test point
    # change colour of data depending on prediction for test point
    axs[0].scatter(x_1[:, 0], x_1[:, 1], alpha=0.5, s=4, color='C3' if class_prediction == 0 else'C0', label='Class 1')
    axs[0].scatter(x_2[:, 0], x_2[:, 1], alpha=0.5, s=4, color='C3' if class_prediction == 1 else'C1', label='Class 2')
    axs[0].scatter(x_3[:, 0], x_3[:, 1], alpha=0.5, s=4, color='C3' if class_prediction == 2 else'C2', label='Class 3')
    axs[0].scatter(test_point[0], test_point[1], s=50, color='black', label='Test Point')
    
    axs[0].set_xlim([-6, 2])
    axs[0].set_ylim([-1.5, 4])
    axs[0].legend(fontsize=14, scatterpoints=3)
    
    # plot bar chart of probabilities assigned to each class
    axs[1].bar([1, 2, 3], [prob_class_1, prob_class_2, prob_class_3], width=0.1, color='grey')
    axs[1].set_xticks([1, 2, 3])
    axs[1].set_ylim([0, 1])
    axs[1].set_title(
        'Prediction is class {:}, with probability {:.2f}'.format(class_prediction + 1, prediction_probability),
        fontsize=16
    )

    plt.tight_layout()

# allow test point to be moved around the plane
interact(plot_classification_behaviour, test_x_1=(-5, 2, 0.01), test_x_2=(-1, 3, 0.01));

interactive(children=(FloatSlider(value=-2.5, description='test_x_1', max=2.0, min=-5.0, step=0.01), FloatSlid…